# Visualize Firm Nodes Edges

Visualize network of company mentions

This notebook aggregates mentions at the firm level

In [1]:
# Packages
import pandas as pd
import os
# Network stuff
import networkx as nx
from pyvis.network import Network

## Load Data

In [2]:
# Load '../../../Data/Company_Mentions/Company_Mentions_With_Ticker.xlsx'
company_mentions_with_ticker = pd.read_excel('../../../Data/Company_Mentions/Company_Mentions_With_Ticker.xlsx')
company_mentions_with_ticker

,ticker,fixed_quarter_date,company_mentioned,count,matched_ticker,Corporation_clean
0,LPX,2016-04-01,MOODY'S,1,MCO,MOODY'S
1,LYV,2015-07-01,APPLE,1,AAPL,APPLE
2,LYV,2016-01-01,APPLE,1,AAPL,APPLE
3,LYV,2016-04-01,APPLE,1,AAPL,APPLE
4,LYV,2016-07-01,CISCO,3,CSCO,CISCO
...,...,...,...,...,...,...
3330,HOV,2015-10-01,MERITAGE HOMES,1,MTH,MERITAGE HOMES
3331,HOV,2015-10-01,CONSOLIDATED,1,CNSL,CONSOLIDATED
3332,KODK,2015-10-01,AMAZON,1,AMZN,AMAZON
3333,KODK,2016-04-01,ENTERPRISE,1,EPD,ENTERPRISE


## Get pairwise tickers

In [3]:
pairwise_df = (company_mentions_with_ticker[['ticker', 'matched_ticker']].rename(columns={'ticker': 'ticker1', 'matched_ticker': 'ticker2'})
                                                                         .value_counts()
                                                                         .reset_index()
                                                                         .rename(columns={0: 'count'}))

pairwise_df

,ticker1,ticker2,count
0,RS,MSCI,26
1,OTEX,ORCL,26
2,EQIX,MSFT,24
3,OTEX,MSFT,24
4,MDLZ,SBUX,22
...,...,...,...
1288,MMM,DHR,1
1289,MMM,HON,1
1290,MMP,EPD,1
1291,MMP,HEP,1


In [4]:
# Order doesn't matter!

# Iterate over rows, create sorted list of tickers
pairwise_df['sorted_tickers'] = pairwise_df[['ticker1', 'ticker2']].apply(lambda x: sorted(x), axis=1)

# Sort the rows by the sorted_tickers column
pairwise_df = pairwise_df.sort_values('sorted_tickers')

# Duplicates on sorted_tickers
#print(pairwise_df[pairwise_df.duplicated('sorted_tickers')])

# Split sorted tickers into two columns again
pairwise_df[['ticker1', 'ticker2']] = pd.DataFrame(pairwise_df['sorted_tickers'].tolist(), index=pairwise_df.index)

# Print duplicates on ticker1 and ticker2
#print(pairwise_df[pairwise_df.duplicated(['ticker1', 'ticker2'])])

# Collapse to sums of count by ticker1 and ticker2
pairwise_df = pairwise_df.groupby(['ticker1', 'ticker2']).agg({'count': 'sum'}).reset_index()

pairwise_df

,ticker1,ticker2,count
0,AAPL,ACIW,1
1,AAPL,ADP,1
2,AAPL,ADSK,2
3,AAPL,ALGT,2
4,AAPL,ALK,1
...,...,...,...
1223,TGT,WWW,2
1224,TMUS,USM,4
1225,TMUS,VMI,2
1226,TOL,VGR,1


## Add Company Name and Sector

In [5]:
# Load rating datasets for company name

# Dataset 1
rating_data_1 = pd.read_csv(os.path.expanduser('~/Box/STAT 222 Capstone/Raw Data/Supplementary Credit Rating Data From Kaggle/corporateCreditRatingWithFinancialRatios.csv'))
print(rating_data_1.columns)
# Keep Ticker and Corporation, rename Ticker to ticker, Corporation to company
rating_data_1 = rating_data_1[['Ticker', 'Corporation']].rename(columns={'Ticker': 'ticker', 'Corporation': 'company'})
# Drop duplicates
rating_data_1.drop_duplicates(inplace=True)

# Dataset 2
rating_data_2 = pd.read_csv(os.path.expanduser('~/Box/STAT 222 Capstone/Raw Data/Credit Rating Data From Kaggle/corporate_rating.csv'))
print(rating_data_2.columns)
# Keep Symbol and Name, rename Symbol to ticker, Name to company
rating_data_2 = rating_data_2[['Symbol', 'Name']].rename(columns={'Symbol': 'ticker', 'Name': 'company'})
# Drop duplicates
rating_data_2.drop_duplicates(inplace=True)

# Stack the two datasets and take a random item for each duplicate on ticker
rating_data_names = pd.concat([rating_data_1, rating_data_2], axis=0)
rating_data_names.drop_duplicates(subset='ticker', keep='first', inplace=True)
rating_data_names


Index(['Rating Agency', 'Corporation', 'Rating', 'Rating Date', 'CIK',
       'Binary Rating', 'SIC Code', 'Sector', 'Ticker', 'Current Ratio',
       'Long-term Debt / Capital', 'Debt/Equity Ratio', 'Gross Margin',
       'Operating Margin', 'EBIT Margin', 'EBITDA Margin',
       'Pre-Tax Profit Margin', 'Net Profit Margin', 'Asset Turnover',
       'ROE - Return On Equity', 'Return On Tangible Equity',
       'ROA - Return On Assets', 'ROI - Return On Investment',
       'Operating Cash Flow Per Share', 'Free Cash Flow Per Share'],
      dtype='object')
Index(['Rating', 'Name', 'Symbol', 'Rating Agency Name', 'Date', 'Sector',
       'currentRatio', 'quickRatio', 'cashRatio', 'daysOfSalesOutstanding',
       'netProfitMargin', 'pretaxProfitMargin', 'grossProfitMargin',
       'operatingProfitMargin', 'returnOnAssets', 'returnOnCapitalEmployed',
       'returnOnEquity', 'assetTurnover', 'fixedAssetTurnover',
       'debtEquityRatio', 'debtRatio', 'effectiveTaxRate',
       'freeCashFl

,ticker,company
0,AWR,American States Water Co.
1,ADP,Automatic Data Processing Inc.
2,AVT,Avnet Inc.
3,CWT,California Water Service Co.
4,CAH,Cardinal Health Inc.
...,...,...
2019,IRS,IRSA Inversiones Y Representaciones S.A.
2020,IT,"Gartner, Inc."
2022,XPER,Xperi Holding Corporation
2023,IMO,Imperial Oil Limited


In [6]:
# Need to reload all data, just ticker and sector columns
# list of files in '../../../Data/All_Data/All_Data_Fixed_Quarter_Dates'
file_list = [f for f in os.listdir(r'../../../Data/All_Data/All_Data_Fixed_Quarter_Dates') if f.endswith('.parquet')]
# read in all parquet files
sector_df = pd.concat([pd.read_parquet(r'../../../Data/All_Data/All_Data_Fixed_Quarter_Dates/' + f, columns=['ticker', 'Sector']) for f in file_list]).drop_duplicates()
sector_df

,ticker,Sector
0,AAPL,Information Technology
10,ABB,Industrials
14,ABBV,Health Care
23,ABC,Health Care
46,ABG,Consumer Discretionary
...,...,...
772,XRAY,Health Care
794,XYL,Industrials
795,YUM,Consumer Discretionary
815,ZBRA,Information Technology


In [7]:
# Inner join rating_data_names and sector_df on ticker
rating_data_names_sector = rating_data_names.merge(sector_df, on='ticker', how='inner')
rating_data_names_sector

,ticker,company,Sector
0,AWR,American States Water Co.,Utilities
1,ADP,Automatic Data Processing Inc.,Information Technology
2,AVT,Avnet Inc.,Information Technology
3,CWT,California Water Service Co.,Utilities
4,CAH,Cardinal Health Inc.,Health Care
...,...,...,...
494,RFP,Resolute Forest Products Inc.,Materials
495,ALKS,Alkermes plc,Health Care
496,EA,Electronic Arts Inc.,Communication Services
497,MX,MagnaChip Semiconductor Corporation,Information Technology


In [8]:
# Left join on ticker1
pairwise_df = pairwise_df.merge(rating_data_names_sector, left_on='ticker1', right_on='ticker', how='left')
pairwise_df.drop(columns='ticker', inplace=True)
pairwise_df.rename(columns={'company': 'company1', 'Sector': 'Sector1'}, inplace=True)
# Left join on ticker2
pairwise_df = pairwise_df.merge(rating_data_names_sector, left_on='ticker2', right_on='ticker', how='left')
pairwise_df.drop(columns='ticker', inplace=True)
pairwise_df.rename(columns={'company': 'company2', 'Sector': 'Sector2'}, inplace=True)
# Put variable names containing 1 first, then 2, then count
pairwise_df = pairwise_df[['ticker1', 'company1', 'Sector1', 'ticker2', 'company2', 'Sector2', 'count']]
# Number of companies
print('number companies')
print(len(set(list(pairwise_df['ticker1'].unique()) + list(pairwise_df['ticker2'].unique()))))
pairwise_df

number companies
387


,ticker1,company1,Sector1,ticker2,company2,Sector2,count
0,AAPL,Apple Inc.,Information Technology,ACIW,ACI Worldwide Inc.,Information Technology,1
1,AAPL,Apple Inc.,Information Technology,ADP,Automatic Data Processing Inc.,Information Technology,1
2,AAPL,Apple Inc.,Information Technology,ADSK,"Autodesk, Inc.",Information Technology,2
3,AAPL,Apple Inc.,Information Technology,ALGT,Allegiant Travel Company,Industrials,2
4,AAPL,Apple Inc.,Information Technology,ALK,Alaska Air Group Inc.,Industrials,1
...,...,...,...,...,...,...,...
1223,TGT,Target Corp.,Consumer Discretionary,WWW,Wolverine World Wide Inc,Consumer Discretionary,2
1224,TMUS,T-Mobile US Inc.,Communication Services,USM,United States Cellular Corporation,Communication Services,4
1225,TMUS,T-Mobile US Inc.,Communication Services,VMI,Valmont Industries Inc.,Industrials,2
1226,TOL,Toll Brothers Inc.,Consumer Discretionary,VGR,Vector Ltd.,Consumer Staples,1


## Plotting

In [9]:
# Keep company1, company2, cosine_similarity
nw_cols = pairwise_df[['company1', 'company2', 'ticker1', 'ticker2', 'count']]
# Rename to 'Source', 'Target', 'Weight'
nw_cols.rename(columns={'company1': 'Source', 'company2': 'Target', 'count': 'weight'}, inplace=True)
# Where missing, fill company1 with ticker1 and company2 with ticker2
nw_cols['Source'].fillna(nw_cols['ticker1'], inplace=True)
nw_cols['Target'].fillna(nw_cols['ticker2'], inplace=True)
# Add column 'Type' with value 'Undirected'
nw_cols['Type'] = 'Undirected'
# Limit to just node cols
#nw_cols = nw_cols[['Source', 'Target']]
print(nw_cols)

                     Source                              Target ticker1  \
0                Apple Inc.                  ACI Worldwide Inc.    AAPL   
1                Apple Inc.      Automatic Data Processing Inc.    AAPL   
2                Apple Inc.                      Autodesk, Inc.    AAPL   
3                Apple Inc.            Allegiant Travel Company    AAPL   
4                Apple Inc.               Alaska Air Group Inc.    AAPL   
...                     ...                                 ...     ...   
1223           Target Corp.            Wolverine World Wide Inc     TGT   
1224       T-Mobile US Inc.  United States Cellular Corporation    TMUS   
1225       T-Mobile US Inc.             Valmont Industries Inc.    TMUS   
1226     Toll Brothers Inc.                         Vector Ltd.     TOL   
1227  WEC Energy Group Inc.                    Xcel Energy Inc.     WEC   

     ticker2  weight        Type  
0       ACIW       1  Undirected  
1        ADP       1  Undirec

C:\Users\ijyli\AppData\Local\Temp\ipykernel_17348\2372261029.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nw_cols.rename(columns={'company1': 'Source', 'company2': 'Target', 'count': 'weight'}, inplace=True)
C:\Users\ijyli\AppData\Local\Temp\ipykernel_17348\2372261029.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nw_cols['Source'].fillna(nw_cols['ticker1'], inplace=True)
C:\Users\ijyli\AppData\Local\Temp\ipykernel_17348\2372261029.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [10]:
# Load as graph
G = nx.from_pandas_edgelist(nw_cols, 
                            source='Source', 
                            target='Target', 
                            edge_attr='weight'
                            )

In [11]:
# Pyvis network
pvn = Network(notebook=True)
pvn.from_nx(G)
#pvn.toggle_physics(True) # turn off movement
# add physics controls
pvn.show_buttons(filter_=['physics'])
# use repulsion
pvn.repulsion(node_distance=100)
pvn.show('../../../Output/Company Mentions Network/Firm_Company_Mentions_Network.html')

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


## Version Sampling 50% of Nodes

In [12]:
# Get all values of ticker across ticker1 and ticker2
all_tickers = list(set(list(pairwise_df['ticker1'].unique()) + list(pairwise_df['ticker2'].unique())))
# Set seed
import random
random.seed(222)
# Sample 50% of tickers
sample_tickers = random.sample(all_tickers, int(0.5 * len(all_tickers)))


In [13]:
# Create nw_cols_sample as items with ticker1 and ticker2 in sample_tickers
nw_cols_sample = nw_cols[(nw_cols['ticker1'].isin(sample_tickers)) & (nw_cols['ticker2'].isin(sample_tickers))]
nw_cols_sample

,Source,Target,ticker1,ticker2,weight,Type
1,Apple Inc.,Automatic Data Processing Inc.,AAPL,ADP,1,Undirected
3,Apple Inc.,Allegiant Travel Company,AAPL,ALGT,2,Undirected
5,Apple Inc.,Allegion PLC,AAPL,ALLE,2,Undirected
7,Apple Inc.,Amkor Technology Inc.,AAPL,AMKR,9,Undirected
8,Apple Inc.,Amazon.com Inc.,AAPL,AMZN,1,Undirected
...,...,...,...,...,...,...
1183,Performance Food Group Inc.,Sysco Corporation,PFGC,SYY,2,Undirected
1191,Pioneer Natural Resources Co.,"W&T Offshore, Inc.",PXD,WTI,1,Undirected
1197,"Republic Services, Inc.","Waste Management, Inc.",RSG,WM,1,Undirected
1215,"Spirit Aerosystems Holdings, Inc.",Textron Inc.,SPR,TXT,1,Undirected


In [14]:
# Load as graph
G_sample = nx.from_pandas_edgelist(nw_cols_sample, 
                            source='Source', 
                            target='Target', 
                            edge_attr='weight'
                            )

In [15]:
# Pyvis network
pvn = Network(notebook=True)
pvn.from_nx(G_sample)
#pvn.toggle_physics(True) # turn off movement
# add physics controls
pvn.show_buttons(filter_=['physics'])
# use repulsion
pvn.repulsion(node_distance=100)
pvn.show('../../../Output/Company Mentions Network/Firm_Company_Mentions_Network_50_Percent_Node_Sample.html')

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
